# Webpage Creator

Task:
* Brainstorm ideas for webpages to build (e.g Pizza store, Gaming, Cooking etc)​.
* Prepare the `task` prompt​ with your idea.
* Run the code, and see your webpage!

Extension:
* Make sure the downloaded images off the internet are placed within `images` folder under `coding` folder, and tell Autogen to use those images.


In [ ]:
task: str = """\

A website about cars.

"""
# task: str = "Make a website about coffee."
# task: str = "Make a website about animals, such as the zoo or a safari or anything else."
# task: str = "Make a website related to pizza."
# task: str = "Make a website related to cooking."
# task: str = "Make a blog post about one of your passions."
# task: str = "Make a useful website for high school students looking to apply to university."
# task: str = """I want to a website about my favourite games, complete with a store, blog etc.
# My goals are to have interesting content for users to read, a visually appealing website, and a store where users can buy games.
# I want the website to be easy to navigate and look good on mobile devices.
# """

In [ ]:
coder_system_message: str = """Website Developer.
You are an expert in programming HTML. You are tasked with creating a website for a client.

Your response must only be a single "```html" block, and include all contents of the file.
Do not abbreviate the HTML tags, and ensure that the file is complete and valid.
"""

In [ ]:
critic_system_message: str = """Critic.
You are a helpful assistant highly skilled in evaluating the quality of a given webpage's HTML code by providing a score from 1 (bad) - 10 (good) while providing clear rationale.
YOU MUST CONSIDER HTML BEST PRACTICES for each evaluation. Specifically, you can carefully evaluate the code across the following dimensions
- Bugs (bugs): Are there logic errors, syntax error or typos? If ANY bug exists, the bug score MUST be less than 5.
- Goal compliance (compliance): How well the code meets the specified webpage goals?
- Navigation (navigation): Is the navigation bar easy to use and understand? Does it have a clear hierarchy? No navigation bar means a score of 0. No href links means a score of 0.
- Layout (layout): Is the layout of the webpage appropriate? Is the webpage responsive? Does it look good on mobile devices?
- Aesthetics (aesthetics): Is the webpage visually appealing? Does it have a good color scheme? Does it have a good font? Does it have a good use of whitespace? If there is no CSS, then mark this as 0.
- Images (images): Do images have a max-width so that they do not take up too much space? Make sure that `max-width: 50%` is the largest an image can be.
- Content (content): Suggest improvements to the content. Are more sections needed? Is it worded well?

YOU MUST PROVIDE A SCORE for each of the above dimensions.
{bugs: 0, compliance: 0, navigation: 0, layout: 0, aesthetics: 0, images: 0, content: 0}
Do not suggest code.
Finally, based on the critique above, suggest a concrete list of actions that the coder should take to improve the code.

After your scores and rationale, provide enhancements to the content that can be made.
"""

## Imports

In [ ]:

# %pip install -U pyautogen
# %pip install -U openai

# Maximum number of messages in the chat
MAX_MESSAGES: int = 10

In [ ]:
import os
import autogen
from openai import OpenAI
import base64
from autogen.agentchat import UserProxyAgent, AssistantAgent

In [ ]:
# List of OpenAI API configurations
config_list: list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
)

apikey = config_list[0]["api_key"]
print(f"Using {len(config_list)} OpenAI API configurations")

## Shared Configs


In [ ]:
# For User Proxy Agents
code_execution_config: bool = False

def is_termination_msg(msg: dict) -> bool:
    return "TERMINATE" in msg["content"]


## Construct Agents

In [ ]:
agents: list = []

In [ ]:
coder = AssistantAgent(
    name="Coder",
    system_message=coder_system_message,
    llm_config={
        "config_list": config_list,
        "cache_seed": None,
        "temperature": 0.1,
        "timeout": 600,
    },
)

agents.append(coder)


In [ ]:
critic = AssistantAgent(
    name="Critic",
    system_message=critic_system_message,
    llm_config={
        "config_list": config_list,
        "cache_seed": None,
        "temperature": 0.3,
        "timeout": 600,
    },
)

agents.append(critic)

### User Proxy Agent


In [ ]:
# The User Proxy Agent starts the conversation
user_proxy = UserProxyAgent(
    name="user_proxy",
    system_message="A human admin.",
    human_input_mode="NEVER",
    is_termination_msg=is_termination_msg,
    code_execution_config=code_execution_config,
    default_auto_reply="",
    max_consecutive_auto_reply=MAX_MESSAGES,
)


### Create Group Chat

In [ ]:
print("Chat participants:")
for agent in agents:
    print(f" - {agent.name} ({agent.__module__.split('.')[-1]})")


In [ ]:
groupchat = autogen.GroupChat(
    agents=agents,
    messages=[],
    speaker_selection_method="round_robin",
    allow_repeat_speaker=False,
    max_round=MAX_MESSAGES,
)

groupchatmanager = autogen.GroupChatManager(
    name="chat_manager",
    groupchat=groupchat,
    is_termination_msg=is_termination_msg,
    llm_config={
        "config_list": config_list,
        "cache_seed": None,
        "temperature": 0.2,
        "timeout": 600,
    },
)


## Setup Conversation

In [ ]:

image_prompt = f"""\
Light, scenic, background for the website: {task}
"""


In [ ]:

client = OpenAI(
    # This is the default and can be omitted
    api_key=apikey,
)
model = "dall-e-3"

 # Generate an image based on the prompt
response = client.images.generate(prompt=image_prompt, model=model,quality="standard", response_format="b64_json", size="1024x1024")

img_data =  response.data[0].b64_json
img_bytes = base64.b64decode(img_data)

with open(os.path.join('coding', 'images', 'background.png'), 'wb') as handler:
    handler.write(img_bytes)


In [ ]:
# Find images that can be used
image_filenames = [os.path.join("images", image)
                   for image in os.listdir(os.path.join("coding", "images"))
                   if image.endswith((".jpg", ".jpeg", ".png"))]


In [ ]:
system_prompt: str = f"""\
Build a responsive HTML website for the client.

Client's request:

```
{task.strip()}
```
"""

images_list_str = '\n'.join(image_filenames)
if images_list_str:
    system_prompt += f"""
Here are some stock photos that are available for use:
{images_list_str}

Be selective with the images you use. You do not need to use them at all.
Use the background.png image as the background for the website.
"""

## Start Conversation

In [ ]:
user_proxy.initiate_chat(
    groupchatmanager,
    message=system_prompt,
)

## Save Webpage


In [ ]:
webpage_content: str = """\
<!DOCTYPE html>
<html>
<head>
    <title>Not Found</title>
</head>
<body>
    <p>HTML could not be extracted from the chat.</p>
</body>
"""

def extract_html_from_message(message: str) -> str:
        start_delimiter = "```html"
        end_delimiter = "```"
        start_index = message.find(start_delimiter)
        end_index = message.find(end_delimiter, start_index + len(start_delimiter))
        return message[start_index + len(start_delimiter):end_index]


for message in groupchat.messages:
        if "```html" in message["content"]:
            webpage_content = extract_html_from_message(message["content"])

In [ ]:
webpage_path: str = os.path.join("coding", "my_webpage.html")

with open(webpage_path, "w") as f:
    f.write(webpage_content)
    print(f"Webpage created at `{webpage_path}`")


## Open Webpage in browser


In [ ]:
# Open webpage in browser
import webbrowser
webbrowser.open(webpage_path, new=1)

# Alternate method to open webpage in browser
# import os
# os.system(f"open {webpage_path}")

# Alternate method 2 to open webpage in browser
# import subprocess
# subprocess.run(["open", webpage_path])


In [ ]:
print("DONE")
